<a href="https://colab.research.google.com/github/joshdsolis/DS-Unit-2-Sprint-4-Model-Validation/blob/master/DS_Unit_2_Sprint_Challenge_4_Model_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Data Science Unit 2 Sprint Challenge 4 — Model Validation

Follow the instructions for each numbered part to earn a score of 2. See the bottom of the notebook for a list of ways you can earn a score of 3.

## Predicting Blood Donations

Our dataset is from a mobile blood donation vehicle in Taiwan. The Blood Transfusion Service Center drives to different universities and collects blood as part of a blood drive.

The goal is to predict the last column, whether the donor made a donation in March 2007, using information about each donor's history. We'll measure success using recall score as the model evaluation metric.

Good data-driven systems for tracking and predicting donations and supply needs can improve the entire supply chain, making sure that more patients get the blood transfusions they need.

#### Run this cell to load the data:

In [0]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/blood-transfusion/transfusion.data')

df = df.rename(columns={
    'Recency (months)': 'months_since_last_donation', 
    'Frequency (times)': 'number_of_donations', 
    'Monetary (c.c. blood)': 'total_volume_donated', 
    'Time (months)': 'months_since_first_donation', 
    'whether he/she donated blood in March 2007': 'made_donation_in_march_2007'
})

In [2]:
df.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


## Part 1.1 — Begin with baselines

What **accuracy score** would you get here with a **"majority class baseline"?** 
 
(You don't need to split the data into train and test sets yet. You can answer this question either with a scikit-learn function or with a pandas function.)

In [3]:
import numpy as np
from sklearn.metrics import accuracy_score

# Making X and y dfs
X = df.drop(columns='made_donation_in_march_2007')
y=df['made_donation_in_march_2007']

# Making majority class for mode
majority_class = y.mode()[0]
y_pred = np.full(shape=y.shape, fill_value=majority_class)

# Accuracy score
accuracy_score(y,y_pred)

0.7620320855614974

What **recall score** would you get here with a **majority class baseline?**

(You can answer this question either with a scikit-learn function or with no code, just your understanding of recall.)

In [4]:
from sklearn.metrics import recall_score
recall_score(y, y_pred)

0.0

## Part 1.2 — Split data

In this Sprint Challenge, you will use "Cross-Validation with Independent Test Set" for your model evaluation protocol.

First, **split the data into `X_train, X_test, y_train, y_test`**, with random shuffle. (You can include 75% of the data in the train set, and hold out 25% for the test set.)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, shuffle=True)

## Part 2.1 — Make a pipeline

Make a **pipeline** which includes:
- Preprocessing with any scikit-learn [**Scaler**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)
- Feature selection with **[`SelectKBest`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html)([`f_classif`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html))**
- Classification with [**`LogisticRegression`**](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [0]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import f_classif, SelectKBest


pipeline = make_pipeline(
    RobustScaler(),
    SelectKBest(f_classif),
    LogisticRegression(solver='lbfgs'))

## Part 2.2 — Do Grid Search Cross-Validation

Do [**GridSearchCV**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) with your pipeline. Use **5 folds** and **recall score**.

Include these **parameters for your grid:**

#### `SelectKBest`
- `k : 1, 2, 3, 4`

#### `LogisticRegression`
- `class_weight : None, 'balanced'`
- `C : .0001, .001, .01, .1, 1.0, 10.0, 100.00, 1000.0, 10000.0`


**Fit** on the appropriate data.

In [28]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'selectkbest__k': [1,2,3,4],
    'logisticregression__class_weight': [None, 'balanced'],
    'logisticregression__C' : [.0001,.001,.01,.1,1.0,10.0,100.00,1000.0,10000.0]
}

gs = GridSearchCV(pipeline, param_grid=param_grid,cv=5,
                 scoring='recall', verbose=1)
gs.fit(X_train, y_train)
validation_score = gs.best_score_

print('Cross-Validation Score: ', -validation_score)
print()
print('Best estimator:', gs.best_estimator_)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Cross-Validation Score:  -0.7804338000416432

Best estimator: Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k=2, score_func=<function f_classif at 0x7ff5b6e889d8>)), ('logisticregression', LogisticRegression(C=0.1, class_weight='balanced', dual=False,
  ...enalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))])


[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:    3.6s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## Part 3 — Show best score and parameters

Display your **best cross-validation score**, and the **best parameters** (the values of `k, class_weight, C`) from the grid search.

(You're not evaluated here on how good your score is, or which parameters you find. You're only evaluated on being able to display the information. There are several ways you can get the information, and any way is acceptable.)

In [33]:
print('Cross-Validation Score: ', -validation_score)
print()
print('Best estimator:', gs.best_estimator_)

Cross-Validation Score:  -0.7804338000416432

Best estimator: Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k=2, score_func=<function f_classif at 0x7ff5b6e889d8>)), ('logisticregression', LogisticRegression(C=0.1, class_weight='balanced', dual=False,
  ...enalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))])


## Part 4 — Calculate classification metrics from a confusion matrix

Suppose this is the confusion matrix for your binary classification model:

<table>
  <tr>
    <th colspan="2" rowspan="2"></th>
    <th colspan="2">Predicted</th>
  </tr>
  <tr>
    <th>Negative</th>
    <th>Positive</th>
  </tr>
  <tr>
    <th rowspan="2">Actual</th>
    <th>Negative</th>
    <td>85</td>
    <td>58</td>
  </tr>
  <tr>
    <th>Positive</th>
    <td>8</td>
    <td>36</td>
  </tr>
</table>

Calculate accuracy

In [34]:
(36+85)/(36+85+58+8)

0.6470588235294118

Calculate precision

In [35]:
36/(36+58)

0.3829787234042553

Calculate recall

In [36]:
36/(36+8)

0.8181818181818182

## BONUS — How you can earn a score of 3

### Part 1
Do feature engineering, to try improving your cross-validation score.

### Part 2
Add transformations in your pipeline and parameters in your grid, to try improving your cross-validation score.

### Part 3
Show names of selected features. Then do a final evaluation on the test set — what is the test score?

### Part 4
Calculate F1 score and False Positive Rate. 

In [37]:
df.head()

,months_since_last_donation,number_of_donations,total_volume_donated,months_since_first_donation,made_donation_in_march_2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [0]:
df['total_volume_donated_sq'] = df['total_volume_donated']**2
df['donations_per_month'] = df['months_since_first_donation']/df['number_of_donations']

In [42]:
import numpy as np
from sklearn.metrics import accuracy_score

# Making X and y dfs
X = df.drop(columns='made_donation_in_march_2007')
y=df['made_donation_in_march_2007']

# Making majority class for mode
majority_class = y.mode()[0]
y_pred = np.full(shape=y.shape, fill_value=majority_class)

# Accuracy score
accuracy_score(y,y_pred)

0.7620320855614974

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, shuffle=True)

In [0]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.linear_model import Ridge


pipeline = make_pipeline(
    RobustScaler(),
    SelectKBest(f_regression),
    LogisticRegression(solver='lbfgs'))

In [55]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'selectkbest__k': range(1, len(X_train.columns)+1),
    'logisticregression__class_weight': [None, 'balanced'],
    'logisticregression__C' : [.0001,.001,.01,.1,1.0,10.0,100.00,1000.0,10000.0]
}

gs = GridSearchCV(pipeline, param_grid=param_grid,cv=5,
                 scoring='recall', verbose=1)
gs.fit(X_train, y_train)
validation_score = gs.best_score_

print('Cross-Validation Score: ', -validation_score)
print()
print('Best estimator:', gs.best_estimator_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Cross-Validation Score:  -0.7865279237828255

Best estimator: Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k=2, score_func=<function f_regression at 0x7ff5b6e88b70>)), ('logisticregression', LogisticRegression(C=0.001, class_weight='balanced', dual=Fals...enalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False))])


[Parallel(n_jobs=1)]: Done 540 out of 540 | elapsed:    6.5s finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [57]:
selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X_train.columns
selected_mask = selector.get_support()
selected_names=all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
  print(name)
  
print()
print("Features not selected:")
for name in unselected_names:
  print(name)

Features selected:
months_since_last_donation
donations_per_month

Features not selected:
number_of_donations
total_volume_donated
months_since_first_donation
total_volume_donated_sq


In [58]:
y_pred = gs.predict(X_test)

test_score = recall_score(y_test, y_pred)

print('Test Score:', test_score)

Test Score: 0.8297872340425532


In [60]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.51      0.65       140
           1       0.36      0.83      0.50        47

   micro avg       0.59      0.59      0.59       187
   macro avg       0.63      0.67      0.58       187
weighted avg       0.76      0.59      0.61       187



In [61]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[71, 69],
       [ 8, 39]])

In [63]:
print('False positive rate:', 69/(71+69))

False positive rate: 0.4928571428571429
